##Evaluación de modelos de clasificación para diagnosticos médicos

In [ ]:
#Librerias

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Librerias para ML
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import *

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
import os
from google.colab import drive

path = "/content/drive/My Drive/"
drive.mount('/content/drive')
os.chdir(path)
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


##Data Set solo con el preprocesado de la primera entrega .

In [ ]:
d=pd.read_csv("IA 1/PYP/Filtrado_20240312.csv", sep=",")
print(d.head(5))
print(d.shape)
#print(d.shape)

     Departamento   Año TipoAtencion  \
0  05 - Antioquia  2021    CONSULTAS   
1  05 - Antioquia  2021    CONSULTAS   
2  05 - Antioquia  2021    CONSULTAS   
3  05 - Antioquia  2021    CONSULTAS   
4  05 - Antioquia  2021    CONSULTAS   

                                         Diagnostico  NumeroAtenciones  
0  E02X - HIPOTIROIDISMO SUBCLINICO POR DEFICIENC...                 1  
1  F069 - TRASTORNO MENTAL NO ESPECIFICADO DEBIDO...                 1  
2     F412 - TRASTORNO MIXTO DE ANSIEDAD Y DEPRESION                 1  
3                        F510 - INSOMNIO NO ORGANICO                 1  
4  F529 - DISFUNCION SEXUAL, NO OCASIONADA POR TR...                 1  
(3567656, 5)


##Valores de la columna tipo de atención

In [ ]:
valores_tipo_atencion= d['TipoAtencion'].unique()
print(valores_tipo_atencion)

['CONSULTAS' 'HOSPITALIZACIONES' 'PROCEDIMIENTOS DE SALUD' 'URGENCIAS']


##Valores de la columna Diagnostico sin preprocesado

In [ ]:
valores_tipo_diagnostico= d['Diagnostico'].unique()
print(valores_tipo_diagnostico.shape)



(11873,)


##Preprocesado para tomar solo los valores con un numero alto de atenciones, ya que hay mucas filas o registros con una sola atencion medica.

In [ ]:

diagnostico_count = d['Diagnostico'].value_counts()
## filtro
#diagnosticos_mas_comunes=diagnostico_count[diagnostico_count>3500].index
## filtrar el dataset  para mantener solo las filas con los diagnosticos mas comunes
#d_filtrado= d[d['Diagnostico'].isin(diagnosticos_mas_comunes)]

d_filtrado= d[d['NumeroAtenciones']>10000]
d_filtrado= d_filtrado.reset_index(drop=True)


print(d_filtrado.shape)
d_filtrado.head(10)


(3730, 5)


,Departamento,Año,TipoAtencion,Diagnostico,NumeroAtenciones
0,05 - Antioquia,2021,CONSULTAS,Z718 - OTRAS CONSULTAS ESPECIFICADAS,88373
1,05 - Antioquia,2021,PROCEDIMIENTOS DE SALUD,R448 - OTROS SINTOMAS Y SIGNOS QUE INVOLUCRAN ...,33040
2,05 - Antioquia,2021,PROCEDIMIENTOS DE SALUD,R104 - OTROS DOLORES ABDOMINALES Y LOS NO ESPE...,23071
3,05 - Antioquia,2021,PROCEDIMIENTOS DE SALUD,1 - NO DEFINIDO,22112
4,05 - Antioquia,2021,PROCEDIMIENTOS DE SALUD,1 - NO DEFINIDO,27592
5,05 - Antioquia,2021,PROCEDIMIENTOS DE SALUD,R448 - OTROS SINTOMAS Y SIGNOS QUE INVOLUCRAN ...,15268
6,05 - Antioquia,2021,PROCEDIMIENTOS DE SALUD,Z108 - OTROS CONTROLES GENERALES DE SALUD DE R...,12894
7,05 - Antioquia,2021,CONSULTAS,R688 - OTROS SINTOMAS Y SIGNOS GENERALES ESPEC...,16549
8,05 - Antioquia,2021,CONSULTAS,R51X - CEFALEA,36271
9,05 - Antioquia,2021,PROCEDIMIENTOS DE SALUD,I10X - HIPERTENSION ESENCIAL (PRIMARIA),13592


##Data Set despues de filtrar, para quedarnos solo con los registres que tienen los diagnosticos con mayor frecuencia.

In [ ]:
d_filtrado


,Departamento,Año,TipoAtencion,Diagnostico,NumeroAtenciones
0,05 - Antioquia,2021,CONSULTAS,Z718 - OTRAS CONSULTAS ESPECIFICADAS,88373
1,05 - Antioquia,2021,PROCEDIMIENTOS DE SALUD,R448 - OTROS SINTOMAS Y SIGNOS QUE INVOLUCRAN ...,33040
2,05 - Antioquia,2021,PROCEDIMIENTOS DE SALUD,R104 - OTROS DOLORES ABDOMINALES Y LOS NO ESPE...,23071
3,05 - Antioquia,2021,PROCEDIMIENTOS DE SALUD,1 - NO DEFINIDO,22112
4,05 - Antioquia,2021,PROCEDIMIENTOS DE SALUD,1 - NO DEFINIDO,27592
...,...,...,...,...,...
3725,81 - Arauca,2021,PROCEDIMIENTOS DE SALUD,1 - NO DEFINIDO,218385
3726,81 - Arauca,2021,PROCEDIMIENTOS DE SALUD,Z000 - EXAMEN MEDICO GENERAL,10964
3727,81 - Arauca,2021,PROCEDIMIENTOS DE SALUD,1 - NO DEFINIDO,73267
3728,81 - Arauca,2021,PROCEDIMIENTOS DE SALUD,1 - NO DEFINIDO,101039


In [ ]:
#Evidencia  del Decision tree sin filtrar:

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import *
### Separar el Ground Truth y el DataSet

y = d['Diagnostico']
X = d[['Departamento', 'Año', 'TipoAtencion','NumeroAtenciones']]



## Codificar variables categoricas a numericas

X_encoded = pd.get_dummies(X)


## Dividir los datos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=21,shuffle=True)


## Entrenando el modelo

dt_classifier = DecisionTreeClassifier()

# Entrenar el modelo
dt_classifier.fit(X_train, y_train)

# Predecir con el conjunto de prueba
y_pred = dt_classifier.predict(X_test)



# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

KeyboardInterrupt: 

##Utilizar el clasificador DecisionTreeClassifier con parámetros por defecto, y reportar accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import *
### Separar el Ground Truth y el DataSet

y = d_filtrado['Diagnostico']
X = d_filtrado[['Departamento', 'Año', 'TipoAtencion','NumeroAtenciones']]



## Codificar variables categoricas a numericas

X_encoded = pd.get_dummies(X)


## Dividir los datos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=21,shuffle=True)


## Entrenando el modelo

dt_classifier = DecisionTreeClassifier()

# Entrenar el modelo
dt_classifier.fit(X_train, y_train)

# Predecir con el conjunto de prueba
y_pred = dt_classifier.predict(X_test)



# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.5536193029490617

Classification Report:
                                                                                                                    precision    recall  f1-score   support

                                                                                                  1 - NO DEFINIDO       0.67      0.73      0.70       114
                                                       A084 - INFECCION INTESTINAL VIRAL, SIN OTRA ESPECIFICACION       0.00      0.00      0.00         0
                                                                               A419 - SEPTICEMIA, NO ESPECIFICADA       0.00      0.00      0.00         1
                        B24X - ENFERMEDAD POR VIRUS DE LA INMUNODEFICIENCIA HUMANA (VIH), SIN OTRA ESPECIFICACION       0.22      0.50      0.31         4
                                                   B342 - INFECCION DEBIDA A CORONAVIRUS, SIN OTRA ESPECIFICACION       0.00      0.00      0.00         2
               

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

#Tunning de parametros  DecisionTreeClassifier : max_depth , criterion  y reporte de accuracy

In [ ]:

y = d_filtrado['Diagnostico']
X = d_filtrado[['Departamento', 'Año', 'TipoAtencion','NumeroAtenciones']]



## Codificar variables categoricas a numericas

X_encoded = pd.get_dummies(X)


## Dividir los datos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=21,shuffle=True)


## Entrenando el modelo

dt_classifier = DecisionTreeClassifier(criterion="gini",max_depth=100)

# Entrenar el modelo
dt_classifier.fit(X_train, y_train)

# Predecir con el conjunto de prueba
y_pred = dt_classifier.predict(X_test)



# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.5576407506702413

Classification Report:
                                                                                                                    precision    recall  f1-score   support

                                                                                                  1 - NO DEFINIDO       0.67      0.73      0.70       114
                                                       A084 - INFECCION INTESTINAL VIRAL, SIN OTRA ESPECIFICACION       0.00      0.00      0.00         0
                                                                               A419 - SEPTICEMIA, NO ESPECIFICADA       0.00      0.00      0.00         1
                        B24X - ENFERMEDAD POR VIRUS DE LA INMUNODEFICIENCIA HUMANA (VIH), SIN OTRA ESPECIFICACION       0.25      0.50      0.33         4
                                                   B342 - INFECCION DEBIDA A CORONAVIRUS, SIN OTRA ESPECIFICACION       0.00      0.00      0.00         2
               

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

##Decision Tree con KrossValidation

In [ ]:
y = d_filtrado['Diagnostico']
X = d_filtrado[['Departamento', 'Año', 'TipoAtencion','NumeroAtenciones']]
X_encoded = pd.get_dummies(X)
y_encoded = y.astype('category').cat.codes
est = DecisionTreeClassifier()

s=cross_val_score(est, X_encoded, y_encoded, cv=KFold(10,shuffle=True),scoring=make_scorer(accuracy_score),error_score="raise")
print(s)
print("Mean Accuracy: %.3f, Std Deviation: %.3f" % (np.mean(s), np.std(s)))

[0.57372654 0.53887399 0.59785523 0.61126005 0.5924933  0.5844504
 0.61662198 0.57640751 0.60857909 0.57908847]
Mean Accuracy: 0.588, Std Deviation: 0.022


In [ ]:
# Asumiendo que d_filtrado es tu DataFrame filtrado
y = d_filtrado['Diagnostico']
X = d_filtrado[['Departamento', 'Año', 'TipoAtencion','NumeroAtenciones']]

# Codificar variables categóricas a numéricas
X_encoded = pd.get_dummies(X)

# Asegurarse de que 'y' esté en el formato adecuado (puede ser necesario si 'y' tiene valores categóricos)
y_encoded = y.astype('category').cat.codes

# Definir el clasificador
est = DecisionTreeClassifier()

# Configurar el Leave-One-Out Cross-Validation
loo = LeaveOneOut()

# Realizar la validación cruzada
s = cross_val_score(est, X_encoded, y_encoded, cv=loo, scoring=make_scorer(accuracy_score), error_score="raise")

# Imprimir resultados
print(s)
print("Mean Accuracy: %.3f, Std Deviation: %.3f" % (np.mean(s), np.std(s)))

[1. 1. 0. ... 1. 1. 1.]
Mean Accuracy: 0.621, Std Deviation: 0.485


## Utilizar el clasificador RandomForestClassifier con parámetros por defecto, y reportar accuracy_score




In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Asumiendo que d_filtrado es tu DataFrame filtrado

X = d_filtrado[['Departamento', 'Año', 'TipoAtencion', 'NumeroAtenciones']]
y = d_filtrado['Diagnostico']

# Codificar variables categóricas a numéricas
X_encoded = pd.get_dummies(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25, random_state=21, shuffle=True)

# Usar RandomForestClassifier con parámetros por defecto
est = RandomForestClassifier()
est.fit(X_train, y_train)

# Predecir con el conjunto de prueba
y_pred_rf = est.predict(X_test)

# Evaluar el modelo
print("Accuracy:", accuracy_score( y_pred_rf,y_test))
print("\nClassification Report:\n", classification_report(y_pred_rf,y_test))

Accuracy: 0.5241157556270096

Classification Report:
                                                                                                                    precision    recall  f1-score   support

                                                                                                  1 - NO DEFINIDO       0.71      0.71      0.71       142
                                                       A084 - INFECCION INTESTINAL VIRAL, SIN OTRA ESPECIFICACION       0.00      0.00      0.00         2
                                                   A09X - DIARREA Y GASTROENTERITIS DE PRESUNTO ORIGEN INFECCIOSO       1.00      0.33      0.50         3
                                                                               A419 - SEPTICEMIA, NO ESPECIFICADA       0.00      0.00      0.00         0
                        B24X - ENFERMEDAD POR VIRUS DE LA INMUNODEFICIENCIA HUMANA (VIH), SIN OTRA ESPECIFICACION       0.67      0.40      0.50        10
               

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

## Tunning de parámetros RandomForestClassifier: n_estimators, max_depth y criterion, y reportar accuracy_score

In [ ]:
X = d_filtrado[['Departamento', 'Año', 'TipoAtencion', 'NumeroAtenciones']]
y = d_filtrado['Diagnostico']

# Codificar variables categóricas a numéricas
X_encoded = pd.get_dummies(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25, random_state=21, shuffle=True)

# Usar RandomForestClassifier con parámetros por defecto
est = RandomForestClassifier(n_estimators=200 , max_depth=40, criterion='log_loss')
est.fit(X_train, y_train)

# Predecir con el conjunto de prueba
y_pred_rf = est.predict(X_test)

# Evaluar el modelo
print("Accuracy:", accuracy_score( y_pred_rf,y_test))
print("\nClassification Report:\n", classification_report(y_pred_rf,y_test))

Accuracy: 0.5241157556270096

Classification Report:
                                                                                                                    precision    recall  f1-score   support

                                                                                                  1 - NO DEFINIDO       0.71      0.71      0.71       143
                                                       A084 - INFECCION INTESTINAL VIRAL, SIN OTRA ESPECIFICACION       0.00      0.00      0.00         2
                                                   A09X - DIARREA Y GASTROENTERITIS DE PRESUNTO ORIGEN INFECCIOSO       1.00      0.33      0.50         3
                                                                               A419 - SEPTICEMIA, NO ESPECIFICADA       0.00      0.00      0.00         0
                        B24X - ENFERMEDAD POR VIRUS DE LA INMUNODEFICIENCIA HUMANA (VIH), SIN OTRA ESPECIFICACION       0.67      0.40      0.50        10
               

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

##Utilizar el clasificador SVC con parámetros por defecto, y con tunning de parámetros

In [ ]:
y = d_filtrado['Diagnostico']
X = d_filtrado[['Departamento', 'Año', 'TipoAtencion','NumeroAtenciones']]

X_encoded = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=21, shuffle=True)


svc_classifier = SVC()
svc_classifier.fit(X_train, y_train)
y_pred_svc = svc_classifier.predict(X_test)

print("SVC Classifier with default parameters")
print("Accuracy:", accuracy_score(y_test, y_pred_svc))
print("\nClassification Report:\n", classification_report(y_test, y_pred_svc))


SVC Classifier with default parameters
Accuracy: 0.15415549597855227

Classification Report:
                                                                                                                    precision    recall  f1-score   support

                                                                                                  1 - NO DEFINIDO       0.15      1.00      0.27       114
                                                                               A419 - SEPTICEMIA, NO ESPECIFICADA       0.00      0.00      0.00         1
                        B24X - ENFERMEDAD POR VIRUS DE LA INMUNODEFICIENCIA HUMANA (VIH), SIN OTRA ESPECIFICACION       0.00      0.00      0.00         4
                                                   B342 - INFECCION DEBIDA A CORONAVIRUS, SIN OTRA ESPECIFICACION       0.00      0.00      0.00         2
                                                                                                B86X - ESCABIOSIS       0.00      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

for kernel in kernels:
    svc_classifier_tuned = SVC(kernel=kernel)
    svc_classifier_tuned.fit(X_train, y_train)
    y_pred_svc_tuned = svc_classifier_tuned.predict(X_test)
    print(f"SVC Classifier with kernel='{kernel}'")
    print("Accuracy:", accuracy_score(y_test, y_pred_svc_tuned))
    print("\nClassification Report:\n", classification_report(y_test, y_pred_svc_tuned))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

# Asumamos que 'd_filtrado' es el DataFrame filtrado con las características y etiquetas
X = d_filtrado.drop('Diagnostico', axis=1)
y = d_filtrado['Diagnostico']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = ['Año', 'NumeroAtenciones']
categorical_features = ['Departamento', 'TipoAtencion']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(11873, activation='relu'))
    model.add(Dense(1, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
epochs_list = [10, 20, 30]
accuracy_scores = []

for epochs in epochs_list:
    model = create_model()
    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=1)
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    print(f'Accuracy for {epochs} epochs: {accuracy}')


Epoch 1/10
94/94 [==============================] - 4s 23ms/step - loss: -105723.0781 - accuracy: 6.7024e-04
Epoch 2/10
94/94 [==============================] - 1s 14ms/step - loss: -2684559.0000 - accuracy: 6.7024e-04
Epoch 3/10
94/94 [==============================] - 1s 14ms/step - loss: -15414014.0000 - accuracy: 6.7024e-04
Epoch 4/10
94/94 [==============================] - 2s 16ms/step - loss: -47082640.0000 - accuracy: 6.7024e-04
Epoch 5/10
94/94 [==============================] - 1s 15ms/step - loss: -105918136.0000 - accuracy: 6.7024e-04
Epoch 6/10
94/94 [==============================] - 2s 19ms/step - loss: -198896672.0000 - accuracy: 6.7024e-04
Epoch 7/10
94/94 [==============================] - 2s 26ms/step - loss: -333046912.0000 - accuracy: 6.7024e-04
Epoch 8/10
94/94 [==============================] - 1s 15ms/step - loss: -513523136.0000 - accuracy: 6.7024e-04
Epoch 9/10
94/94 [==============================] - 2s 24ms/step - loss: -745985984.0000 - accuracy: 6.7024e-04

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

# Asumamos que 'd_filtrado' es el DataFrame filtrado con las características y etiquetas
X = d_filtrado.drop('Diagnostico', axis=1)
y = d_filtrado['Diagnostico']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = ['Año', 'NumeroAtenciones']
categorical_features = ['Departamento', 'TipoAtencion']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(11873, activation='relu'))
    model.add(Dense(11873, activation='relu'))
    model.add(Dense(11873, activation='relu'))
    model.add(Dense(1, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
epochs_list = [10, 20, 30]
accuracy_scores = []

for epochs in epochs_list:
    model = create_model()
    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=1)
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    print(f'Accuracy for {epochs} epochs: {accuracy}')


Epoch 1/10
94/94 [==============================] - 604s 6s/step - loss: -74816946176.0000 - accuracy: 6.7024e-04
Epoch 2/10
94/94 [==============================] - 589s 6s/step - loss: -18140666789888.0000 - accuracy: 6.7024e-04
Epoch 3/10
94/94 [==============================] - 591s 6s/step - loss: -354926232338432.0000 - accuracy: 6.7024e-04
Epoch 4/10
94/94 [==============================] - 588s 6s/step - loss: -2527923798016000.0000 - accuracy: 6.7024e-04
Epoch 5/10
94/94 [==============================] - 589s 6s/step - loss: -10409518961786880.0000 - accuracy: 6.7024e-04
Epoch 6/10
94/94 [==============================] - 594s 6s/step - loss: -31221575416020992.0000 - accuracy: 6.7024e-04
Epoch 7/10
94/94 [==============================] - 593s 6s/step - loss: -76372103435124736.0000 - accuracy: 6.7024e-04
Epoch 8/10
94/94 [==============================] - 597s 6s/step - loss: -161703972506173440.0000 - accuracy: 6.7024e-04
Epoch 9/10
94/94 [==============================] -

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

# Asumamos que 'd_filtrado' es el DataFrame filtrado con las características y etiquetas
X = d_filtrado.drop('Diagnostico', axis=1)
y = d_filtrado['Diagnostico']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = ['Año', 'NumeroAtenciones']
categorical_features = ['Departamento', 'TipoAtencion']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(11873, activation='relu'))
    model.add(Dense(11873, activation='relu'))
    model.add(Dense(11873, activation='relu'))
    model.add(Dense(11873, activation='relu'))
    model.add(Dense(11873, activation='relu'))
    model.add(Dense(11873, activation='relu'))
    model.add(Dense(1, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
epochs_list = [10, 20, 30]
accuracy_scores = []

for epochs in epochs_list:
    model = create_model()
    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=1)
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    print(f'Accuracy for {epochs} epochs: {accuracy}')

Epoch 1/10
17/94 [====>.........................] - ETA: 19:33 - loss: -578358345728.0000 - accuracy: 0.0000e+00

KeyboardInterrupt: 